# PyTorch

In [6]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torchvision import datasets, transforms

In [3]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

# Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [5]:
train_dir = 'C:/Users/신상훈/Desktop/새 폴더/dataset/mnist_png/training/'
test_dir = 'C:/Users/신상훈/Desktop/새 폴더/dataset/mnist_png/testing/'

In [7]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [12]:
train_dataset = datasets.ImageFolder(root = train_dir,
                                    transform = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                    ]))
test_dataset = datasets.ImageFolder(root=test_dir,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))
train_loader = torch.utils.data.DataLoader(train_dataset,
                                         batch_size=2,
                                         shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=2
                                         )

## Optimization

In [13]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

## Training

In [14]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.264696
Train Epoch: 1 [200/60000 (0%)]	Loss: 2.237278
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.195709
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.057271
Train Epoch: 1 [800/60000 (1%)]	Loss: 1.349012
Train Epoch: 1 [1000/60000 (2%)]	Loss: 1.804608
Train Epoch: 1 [1200/60000 (2%)]	Loss: 0.867617
Train Epoch: 1 [1400/60000 (2%)]	Loss: 0.233244
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.869081
Train Epoch: 1 [1800/60000 (3%)]	Loss: 0.299162
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.080674
Train Epoch: 1 [2200/60000 (4%)]	Loss: 0.093477
Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.130369
Train Epoch: 1 [2600/60000 (4%)]	Loss: 0.771551
Train Epoch: 1 [2800/60000 (5%)]	Loss: 0.345080
Train Epoch: 1 [3000/60000 (5%)]	Loss: 0.139626
Train Epoch: 1 [3200/60000 (5%)]	Loss: 3.310799
Train Epoch: 1 [3400/60000 (6%)]	Loss: 1.596446
Train Epoch: 1 [3600/60000 (6%)]	Loss: 0.233228
Train Epoch: 1 [3800/60000 (6%)]	Loss: 0.005435
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.476647

KeyboardInterrupt: 